In [1]:
import requests
from typing import Annotated, Dict, Any
from pydantic import Field


def get_weather(
    location: Annotated[str, Field(description="City or location to fetch weather for.")]
) -> Dict[str, Any]:
    """
    Fetches detailed real-time weather from MET Norway API.
    Returns a JSON object with temperature, humidity, wind, clouds,
    pressure, precipitation, and human-readable condition.
    """
    try:
        # Step 1: Convert city -> lat/lon
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_res = requests.get(geo_url, params={"name": location}, timeout=8)

        if geo_res.status_code != 200 or not geo_res.json().get("results"):
            return {"error": f"Unable to find the location '{location}'."}

        place = geo_res.json()["results"][0]
        lat, lon = place["latitude"], place["longitude"]

        # Step 2: Weather from MET Norway
        weather_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
        headers = {"User-Agent": "your-app-name-here"}

        weather_res = requests.get(
            weather_url,
            params={"lat": lat, "lon": lon},
            headers=headers,
            timeout=8
        )

        if weather_res.status_code != 200:
            return {"error": "Unable to fetch weather at the moment."}

        data = weather_res.json()
        current = data["properties"]["timeseries"][0]["data"]
        instant = current["instant"]["details"]

        # Extract instant values
        temp = instant.get("air_temperature")
        humidity = instant.get("relative_humidity")
        pressure = instant.get("air_pressure_at_sea_level")
        dewpoint = instant.get("dew_point_temperature")
        wind_speed = instant.get("wind_speed")
        wind_gust = instant.get("wind_speed_of_gust")
        wind_dir = instant.get("wind_from_direction")
        cloud_cover = instant.get("cloud_area_fraction")
        fog_area = instant.get("fog_area_fraction")

        # Next 1 hour forecast details
        next_1h = current.get("next_1_hours", {})
        symbol = next_1h.get("summary", {}).get("symbol_code", "unknown")
        precipitation = next_1h.get("details", {}).get("precipitation_amount")

        # Convert symbol code -> readable condition
        condition_text = (
            symbol.replace("_day", "")
                  .replace("_night", "")
                  .replace("_", " ")
                  .title()
        )

        return {
            "location": {
                "name": place["name"],
                "country": place["country"],
                "latitude": lat,
                "longitude": lon
            },
            "current_weather": {
                "condition": condition_text,
                "temperature_c": temp,
                "dew_point_c": dewpoint,
                "humidity_pct": humidity,
                "pressure_hpa": pressure,
                "cloud_cover_pct": cloud_cover,
                "fog_area_pct": fog_area,
                "wind_speed_mps": wind_speed,
                "wind_gust_mps": wind_gust,
                "wind_direction_deg": wind_dir,
                "precipitation_next_1h_mm": precipitation,
                "symbol_code": symbol
            }
        }

    except Exception as e:
        return {"error": str(e)}


In [2]:
get_weather('Pune')

{'location': {'name': 'Pune',
  'country': 'India',
  'latitude': 18.51957,
  'longitude': 73.85535},
 'current_weather': {'condition': 'Clearsky',
  'temperature_c': 14.8,
  'dew_point_c': None,
  'humidity_pct': 63.6,
  'pressure_hpa': 1017.4,
  'cloud_cover_pct': 0.0,
  'fog_area_pct': None,
  'wind_speed_mps': 2.8,
  'wind_gust_mps': None,
  'wind_direction_deg': 112.6,
  'precipitation_next_1h_mm': 0.0,
  'symbol_code': 'clearsky_day'}}

In [3]:
import requests
from typing import Annotated
from pydantic import Field
from agent_framework import ai_function

@ai_function(name="weather_tool", description="Retrieves weather information for any location")
def get_weather(
    location: Annotated[str, Field(description="City or location to fetch weather for.")]
) -> Dict[str, Any]:
    """
    Fetches detailed real-time weather from MET Norway API.
    Returns a JSON object with temperature, humidity, wind, clouds,
    pressure, precipitation, and human-readable condition.
    """
    try:
        # Step 1: Convert city -> lat/lon
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo_res = requests.get(geo_url, params={"name": location}, timeout=8)

        if geo_res.status_code != 200 or not geo_res.json().get("results"):
            return {"error": f"Unable to find the location '{location}'."}

        place = geo_res.json()["results"][0]
        lat, lon = place["latitude"], place["longitude"]

        # Step 2: Weather from MET Norway
        weather_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
        headers = {"User-Agent": "your-app-name-here"}

        weather_res = requests.get(
            weather_url,
            params={"lat": lat, "lon": lon},
            headers=headers,
            timeout=8
        )

        if weather_res.status_code != 200:
            return {"error": "Unable to fetch weather at the moment."}

        data = weather_res.json()
        current = data["properties"]["timeseries"][0]["data"]
        instant = current["instant"]["details"]

        # Extract instant values
        temp = instant.get("air_temperature")
        humidity = instant.get("relative_humidity")
        pressure = instant.get("air_pressure_at_sea_level")
        dewpoint = instant.get("dew_point_temperature")
        wind_speed = instant.get("wind_speed")
        wind_gust = instant.get("wind_speed_of_gust")
        wind_dir = instant.get("wind_from_direction")
        cloud_cover = instant.get("cloud_area_fraction")
        fog_area = instant.get("fog_area_fraction")

        # Next 1 hour forecast details
        next_1h = current.get("next_1_hours", {})
        symbol = next_1h.get("summary", {}).get("symbol_code", "unknown")
        precipitation = next_1h.get("details", {}).get("precipitation_amount")

        # Convert symbol code -> readable condition
        condition_text = (
            symbol.replace("_day", "")
                  .replace("_night", "")
                  .replace("_", " ")
                  .title()
        )

        return {
            "location": {
                "name": place["name"],
                "country": place["country"],
                "latitude": lat,
                "longitude": lon
            },
            "current_weather": {
                "condition": condition_text,
                "temperature_c": temp,
                "dew_point_c": dewpoint,
                "humidity_pct": humidity,
                "pressure_hpa": pressure,
                "cloud_cover_pct": cloud_cover,
                "fog_area_pct": fog_area,
                "wind_speed_mps": wind_speed,
                "wind_gust_mps": wind_gust,
                "wind_direction_deg": wind_dir,
                "precipitation_next_1h_mm": precipitation,
                "symbol_code": symbol
            }
        }

    except Exception as e:
        return {"error": str(e)}


In [4]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

instructions = """
You are a helpful and versatile AI assistant.

You may use tools when needed to provide accurate and complete responses. 
Use your judgment to decide when a tool is appropriate.

When you use a tool, return its output exactly as provided—do not modify, 
summarize, or remove fields from the tool's return value.

When no tool is required, answer normally using your own reasoning and knowledge.

Your goal is to provide clear, correct, and helpful responses across a wide 
range of user queries.
"""


client = AzureOpenAIChatClient(credential=AzureCliCredential())
agent = client.create_agent(
    instructions=instructions,
    tools=get_weather
)

In [5]:
async def main():
    result = await agent.run("What is the weather like in Pune?")
    print(result.text)

await main()

Currently, the weather in Pune is clear with the following details:

- **Condition:** Clear sky
- **Temperature:** 14.8 °C
- **Humidity:** 63.6%
- **Pressure:** 1017.4 hPa
- **Cloud Cover:** 0.0%
- **Wind Speed:** 2.8 m/s (from 112.6°)
- **Precipitation (next 1 hour):** 0.0 mm

It's a pleasant day with no rain expected.


In [6]:
import requests
from typing import Dict, Any, Annotated
from pydantic import Field


class WeatherTools:
    def __init__(self):
        self.last_location = None

    # ============================================================
    # Tool 1 — Get Weather
    # ============================================================
    def get_weather(
        self,
        location: Annotated[str, Field(description="City or location to fetch weather for.")]
    ) -> Dict[str, Any]:

        try:
            self.last_location = location

            # Step 1: Geocoding
            geo_url = "https://geocoding-api.open-meteo.com/v1/search"
            geo_res = requests.get(geo_url, params={"name": location}, timeout=8)

            if geo_res.status_code != 200 or not geo_res.json().get("results"):
                return {"error": f"Unable to find '{location}'."}

            place = geo_res.json()["results"][0]
            lat, lon = place["latitude"], place["longitude"]

            # Step 2: Weather
            weather_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
            headers = {"User-Agent": "weather-agent-demo"}

            weather_res = requests.get(
                weather_url,
                params={"lat": lat, "lon": lon},
                headers=headers,
                timeout=8
            )

            if weather_res.status_code != 200:
                return {"error": "Unable to fetch weather."}

            data = weather_res.json()
            current = data["properties"]["timeseries"][0]["data"]
            instant = current["instant"]["details"]

            # Extract values
            temp = instant.get("air_temperature")
            humidity = instant.get("relative_humidity")
            pressure = instant.get("air_pressure_at_sea_level")
            cloud_cover = instant.get("cloud_area_fraction")
            wind_speed = instant.get("wind_speed")

            next_1h = current.get("next_1_hours", {})
            precipitation = next_1h.get("details", {}).get("precipitation_amount")

            symbol = next_1h.get("summary", {}).get("symbol_code", "unknown")
            condition = (
                symbol.replace("_day", "")
                      .replace("_night", "")
                      .replace("_", " ")
                      .title()
            )

            return {
                "location": place["name"],
                "condition": condition,
                "temperature": temp,
                "humidity": humidity,
                "pressure": pressure,
                "cloud_cover": cloud_cover,
                "wind_speed": wind_speed,
                "precipitation_next_1h": precipitation
            }

        except Exception as e:
            return {"error": str(e)}

    # ============================================================
    # Tool 2 — Recommendation Tool
    # ============================================================
    def get_weather_recommendation(
        self,
        condition: Annotated[str, Field(description="Weather condition text.")],
        temperature: Annotated[float, Field(description="Temperature in °C.")],
        humidity: Annotated[float | None, Field(description="Humidity percent.")] = None,
        cloud_cover: Annotated[float | None, Field(description="Cloud cover percent.")] = None,
        wind_speed: Annotated[float | None, Field(description="Wind speed in m/s.")] = None,
        precipitation_next_1h: Annotated[float | None, Field(description="Precipitation next hour.")] = None
    ) -> Dict[str, Any]:
        """
        Simple recommendation logic based on primitive parameters.
        """

        recommendations = []

        # Temperature
        if temperature < 10:
            recommendations.append("It's cold — wear warm clothing.")
        elif temperature < 20:
            recommendations.append("Cool weather — consider a light jacket.")
        elif temperature > 30:
            recommendations.append("It's hot — stay hydrated.")
        else:
            recommendations.append("Pleasant temperature.")

        # Rain
        if precipitation_next_1h and precipitation_next_1h > 0:
            recommendations.append("Carry an umbrella.")
        else:
            recommendations.append("No rain expected.")

        # Clouds
        if cloud_cover is not None and cloud_cover < 20:
            recommendations.append("Clear skies — great for outdoor plans.")

        # Wind
        if wind_speed is not None and wind_speed > 10:
            recommendations.append("Windy — secure loose items.")

        # Condition logic
        cond = condition.lower()
        if "clear" in cond:
            recommendations.append("Perfect clear sky for outdoor activities.")
        if "cloud" in cond:
            recommendations.append("Cloudy skies — sunlight may be dim.")
        if "rain" in cond:
            recommendations.append("Rainy conditions — roads may be slippery.")

        return {"recommendations": recommendations}


In [7]:
instructions = """
You are a helpful assistant. You may use tools when needed.

When using a tool, pass input parameters exactly as the tool expects and 
never as plain text or summaries. If a previous tool returned structured 
data, use those fields directly as the arguments to the next tool.

Do not rewrite, summarize, or reformat tool inputs or outputs. 
Use tools when appropriate and return their outputs exactly as they provide.
"""

tools = WeatherTools()
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions=instructions,
    tools=[tools.get_weather, tools.get_weather_recommendation],
    enable_trace=True
)

In [8]:
async def main():
    result = await agent.run("What is the weather like in Pune? and suggest the weather recommendation for clothing")
    print(result.text)
    return result

result = await main()

The weather in Pune is currently clear with a temperature of 14.8°C and humidity at 63.6%. The wind speed is 2.8 m/s, and there is no precipitation expected in the next hour.

### Weather Recommendations for Clothing:
- It's cool weather, so consider wearing a light jacket.
- No rain is expected, so you can enjoy outdoor activities comfortably.
- Perfect clear sky for outdoor activities.


In [9]:
from agent_framework._types import FunctionCallContent, FunctionResultContent

for msg in result.messages:
    for content in msg.contents:
        if isinstance(content, FunctionCallContent):
            print("\n Tool Call:")
            print("Tool Name:", content.name)
            print("Arguments:", content.arguments)
            print("------------------------------------------------")


 Tool Call:
Tool Name: get_weather
Arguments: {"location":"Pune"}
------------------------------------------------

 Tool Call:
Tool Name: get_weather_recommendation
Arguments: {"condition":"Clearsky","temperature":14.8,"humidity":63.6,"wind_speed":2.8}
------------------------------------------------


In [11]:
for msg in result.messages:
    for content in msg.contents:
        if isinstance(content, FunctionResultContent):
            print("\nTool Output:")
            print("Output:", content.result)       # REAL tool output
            print("------------------------------------------------")



Tool Output:
Output: {'location': 'Pune', 'condition': 'Clearsky', 'temperature': 14.8, 'humidity': 63.6, 'pressure': 1017.4, 'cloud_cover': 0.0, 'wind_speed': 2.8, 'precipitation_next_1h': 0.0}
------------------------------------------------

Tool Output:
Output: {'recommendations': ['Cool weather — consider a light jacket.', 'No rain expected.', 'Perfect clear sky for outdoor activities.']}
------------------------------------------------
